In [1]:
from google.colab import drive
drive.mount("/F")

Mounted at /F


In [2]:
import pandas as pd

In [3]:
base = pd.read_csv('/F/My Drive/Maratona/Autoencoders/credit_data.csv')

In [4]:
base.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [5]:
base = base.drop('i#clientid', axis = 1)
base.head()

,income,age,loan,c#default
0,66155.925095,59.017015,8106.532131,0
1,34415.153966,48.117153,6564.745018,0
2,57317.170063,63.108049,8020.953296,0
3,42709.534201,45.751972,6103.642260,0
4,66952.688845,18.584336,8770.099235,1


In [7]:
base.shape

(2000, 4)

In [9]:
base = base.dropna()
base.shape

(1997, 4)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
base.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,income,age,loan,c#default
0,1.453898,1.336861,1.201907,0
1,-0.762398,0.536639,0.695744,0
2,0.836733,1.637207,1.173812,0
3,-0.183244,0.362998,0.544366,0
4,1.509532,-1.631534,1.419754,1


In [11]:
X = base.drop('c#default', axis = 1)
y = base['c#default']

In [12]:
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [13]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: c#default, dtype: int64

In [14]:
X.columns

Index(['income', 'age', 'loan'], dtype='object')

In [16]:
import tensorflow as tf

In [17]:
colunas = [tf.feature_column.numeric_column(key = column) for column in X.columns]

In [18]:
print(colunas[2])

NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [44]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [20]:
X_treinamento.shape

(1397, 3)

In [21]:
X_teste.shape

(600, 3)

In [22]:
def make_input_fn(X, y = None, num_epochs = None, shuffle = True, batch_size = 8):
    def input_function():
        # As variáveis previsoras precisam ser passadas na função seguinte como um dicionário com os mesmos nomes definidos na chamada de `tf.feature_column.numeric_column`
        ds = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [23]:
funcao_treinamento = make_input_fn(X_treinamento, y_treinamento)

In [24]:
config = tf.estimator.RunConfig(tf_random_seed = 42)
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4], config = config)
classificador.train(input_fn = funcao_treinamento, steps = 1000)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdeazsfj3', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and grap

In [25]:
funcao_teste = make_input_fn(X_teste, y_teste, num_epochs = 1000, shuffle = False)

In [26]:
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-27T18:12:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdeazsfj3/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Inference Time : 1.45329s
INFO:tensorflow:Finished evaluation at 2021-01-27-18:12:39
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7105, accuracy_baseline = 0.848875, auc = 0.27555847, auc_precision_recall = 0.09800053, average_loss = 0.64567107, global_step

In [36]:
metricas_teste

{'accuracy': 0.7105,
 'accuracy_baseline': 0.848875,
 'auc': 0.27555847,
 'auc_precision_recall': 0.09800053,
 'average_loss': 0.64567107,
 'global_step': 1000,
 'label/mean': 0.151125,
 'loss': 0.64567107,
 'precision': 0.0,
 'prediction/mean': 0.44303054,
 'recall': 0.0}